In [1]:
# -*- coding: utf-8 -*-

##############################################################################
##  Super Composite Likelihood algorithm.
##
##  Copyright 2021 Daniel Wilson and Alejandra Vergara.
##  All rights reserved.
##
##############################################################################

import numpy as np
import pandas as pd
import msprime
import math
import random
import scipy
import seaborn as sns
import tskit
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.colors as mcolors
import warnings
import multiprocessing
import lmfit
import time as time
import scipy.stats as stats
from numpy.random import rand
from scipy.stats import poisson
from scipy.optimize import minimize_scalar, minimize
from lmfit import Minimizer, Parameters, fit_report
from IPython.display import SVG, display, Math

%matplotlib inline

#!msprime --version
plt.rcParams["figure.figsize"] = (11, 5)
warnings.simplefilter(action='ignore', category=FutureWarning)
np.set_printoptions(suppress=True)

In [2]:
### Simulation of coalescent time considering theta and migrations

def simulate_tree_MM(samp_size, L):
    """
    Coalescent simulation with msprime: Population structure
    
    Parameters
    ----------
    Sample_size: List of sequences in the sample. Each sample is independent.
    L : Length of the sequences (integer).
    N: Independent replicates of the coalescent for a sample of n sequences into d demes. 
    Seed: The process to be unique.
        
    Returns
    -------
    Discrete-deme model of population structure in which d panmictic populations
    exchange migrants according to the rates defined in a d × d matrix.
    """

    demography = msprime.Demography()
    demography.add_population(name="deme_1", initial_size=1)
    demography.add_population(name="deme_2", initial_size=1)
    demography.set_symmetric_migration_rate(populations=["deme_1", "deme_2"], rate=0.1/2)
#     demography.set_migration_rate(source="deme_1", dest="deme_2", rate=M/2)
#     demography.set_migration_rate(source="deme_2", dest="deme_1", rate=M/2)
    return msprime.sim_ancestry(samples={"deme_1": samp_size_per_deme, "deme_2": samp_size_per_deme},
    #return msprime.sim_ancestry(samples={"deme_1": samp_size_per_deme, "deme_2": samp_size_per_deme, "deme_3": samp_size_per_deme},
                                demography=demography, 
                                sequence_length=L,
                                #record_migrations=True,
                                #discrete_genome=True,
                                #model="hudson",
                                num_labels=True,
                                ploidy = 1)
#                                 random_seed = 42)

In [3]:
### Simulation of coalescent time 

def simulate_tree_M(samp_size, L):
    """
    Coalescent simulation with msprime: Population structure
    
    Parameters
    ----------
    Sample_size: List of sequences in the sample. Each sample is independent.
    L : Length of the sequences (integer).
    N: Independent replicates of the coalescent for a sample of n sequences into d demes. 
    Seed: The process to be unique.
        
    Returns
    -------
    Discrete-deme model of population structure in which d panmictic populations
    exchange migrants according to the rates defined in a d × d matrix.
    """
#msprime.sim_ancestry
    print("Done")
    return msprime.sim_ancestry(samp_size,
                                sequence_length=L,
                                discrete_genome=True,
                                Ne = 1,
                                ploidy = 1)
    

In [4]:
### Branches calculation 
# where ts -> simulate_tree_MM(samp_size, L)

def calc_branches(ts, theta):
    """
    Branches calculation
    
    Parameters
    ----------
    ts : simulator of coalesce events
        
    Returns
    -------
    Mutations and  over the node IDs in this tree
    """

    ts.dump("big_tree.trees")
    tree = ts.first()
    first_edges = next(ts.edge_diffs())
    node_times = ts.tables.nodes.time
    model = msprime.JC69()
    mut = msprime.sim_mutations(sims_reps, rate=0.01/2, random_seed=1, model=model, keep=True) 
    theta_msprime = mut.diversity()
    mutas = mut.num_mutations
    branch_lengths_above_node = np.zeros(ts.num_nodes)
    branch_load = np.zeros(ts.num_nodes)
   

    for edge in first_edges.edges_in:
        branch_lengths_above_node[edge.id] = node_times[edge.parent] - node_times[edge.child]
        if edge.child != tree.root:
            branch_load[edge.id] = stats.poisson.rvs((theta/2)*branch_lengths_above_node[edge.id])

    #colour map shows in which population samples coalesced
#     colour_map = {0:"red", 1:"blue"}
#     node_colours = {u: colour_map[tree.population(u)] for u in tree.nodes()}
#     display(SVG(tree.draw(format="SVG",width=800, height=400,
#                         node_colours=node_colours)))
    
#     print(branch_lengths_above_node)
    return branch_load, branch_lengths_above_node, theta_msprime

In [5]:
### Marginal probability
### Calculate pairwise distance matrix from tree and # mutations per branch
# where ts -> simulate_tree_MM(samp_size, L) and branch -> calc_branches(ts, theta)

def calc_pi(ts, branch):
    """
    Pi calculation
    Calculate pairwise distance matrix from tree and number of mutations per branch
    
    Parameters
    ----------
    ts : 
        
    Returns
    -------
    Discrete-deme model of population structure in which d panmictic populations
    exchange migrants according to the rates defined in a d × d matrix.
    """

    
    tree = ts.first()
    first_edges = next(ts.edge_diffs())
    node_times = ts.tables.nodes.time
    pi_n = np.zeros((ts.num_nodes,ts.num_nodes))
    pi_d2 = np.zeros((ts.num_nodes,ts.num_nodes))
    pi = np.zeros((samp_size,samp_size))
    
    
    childs_and_parents = {}
    parent_to_childs = {}
    for edge in first_edges.edges_in:
        childs_and_parents[edge.child] = edge.parent
    
    childs_and_ancestors = {}
    for i in childs_and_parents: # child
        j = childs_and_parents[i] #parent
        childs_and_ancestors[i] = [j]
        while j in childs_and_parents:
            j = childs_and_parents[j]
            childs_and_ancestors[i].append(j)
    #parent_to_childs = {v: k for k, v in childs_and_ancestors.items()}

    childs_and_distances2ancestors = {}
    childs_and_dis2anc_mut = {}
    for i in childs_and_ancestors:
        js = childs_and_ancestors[i]
        if edge.child != tree.root:
        #childs_and_distances2ancestors[i] = [node_times[j] - node_times[i] for j in js]
            childs_and_distances2ancestors[i] = [1 for j in js] #sum all mutations up for chil to parents 
            childs_and_dis2anc_mut[i] = [branch[i] for j in js]
            

    for i in childs_and_distances2ancestors:
        j = childs_and_ancestors[i] #parent e.g, 0 to [32, 35, 40, 41, 54, 55, 56, 57, 60, 61]
        d = childs_and_distances2ancestors[i]
        d2 = childs_and_dis2anc_mut[i]
        pi_n[i,j] = d
#         pi_n[j,i] = d 
#         pi_d2[i,j] = d2
    pi_n[np.eye(len(pi_n), k=0, dtype='bool')] = 1
    
#     #### Danny data -mini
#     pi_matrix = pd.read_csv('treemut.csv', sep=",", header='infer')
#     pi_n = np.array(pi_matrix)
    
    
    ### Calculate pairwise distances
    for i in range(samp_size): 
        for j in range(i):
            pi[i,j] = 0
            #for nodes in reversed(range(ts.num_nodes)):
            for nodes in range(ts.num_nodes):
                if(pi_n[i][nodes]!=pi_n[j][nodes]):
                    pi[i,j] += branch[nodes]

    return pi

In [6]:
### Compute pairwise distances between and within populations
# Pairwise probability
# where pi -> calc_pi(ts, branch)

def pairwise_pi_wb(pi, deme_config, ndemes, L):
    
    ## Compute pairwise distances
    n = pi.shape[0]
    pi_wb = np.zeros(5)
    
    piw_num = 0.0
    piw_den = 0.0
    pib_num = 0.0
    pib_den = 0.0
   
    for i in range(n):
        for j in range(i):
            if(deme_config[i] == deme_config[j]):
                piw_num += pi[i,j]
                piw_den += L
            else:
                pib_num += pi[i,j]
                pib_den += L
                

    pi_wb[0] = piw_num
    pi_wb[1] = piw_den
    pi_wb[2] = pib_num
    pi_wb[3] = pib_den
    pi_wb[4] = (pi_wb[0]/pi_wb[1]) + (pi_wb[2]/pi_wb[3])
    
#     print(pi_wb)
    
    return pi_wb 
    

In [7]:
def cll(piw_num, piw_den, pib_num, pib_den, theta, M, ndemes):
    #Compute canonical parameters
    pw = theta * ndemes
    pb = theta * (ndemes + (ndemes-1.0)/M)
    return pi_wb[0] * log(pw) + (pi_wb[1]-pi_wb[0]) * log(1-pw) + pi_wb[2] * log(pb) + (pi_wb[3]-pi_wb[2]) * log(1-pb)

In [8]:
# Observed pairwise composite score (marginal) 
# Derivative wrt theta of piw_num * log(pw) + (piw_den-piw_num) * log(1-pw) + pib_num * log(pb) + (pib_den-pib_num) * log(1-pb)
# Expected value of the gradient of the composite score

def pairwise_score(theta, M, ndemes, piw_num, piw_den, pib_num, pib_den):

    u_1 =(((piw_num-pib_num)/(theta)) + ((ndemes*(piw_den-piw_num))/(theta*ndemes-1)) - 
          (((pib_den-pib_num)*(M*ndemes + (ndemes-1)))/((theta*(M*ndemes+(ndemes-1))) - M)))

    return u_1

In [9]:
# Full-loglikelihood wrt subpopulations
# where pairwise_pi_wb_prob -> pairwise_pi_wb(pi, deme_config, ndemes, L) and pi_wb[4] 

def full_loglikelihood(pairwise_pi_wb_prob):
    fl = (np.log(pairwise_pi_wb_prob)).sum()
    return -fl   

In [10]:
# Expected marginal score per datapoint
# where p_ij is pairwise_pi_wb(pi, deme_config, ndemes, L) -> pi_wb[4]
# and score_der -> pairwise_score(theta, M, ndemes, piw_num, piw_den,...)

def E_mscore(score_der,p_ij):
    exp_mscore = (score_der * p_ij).sum()
    return exp_mscore

In [11]:
# Variance in marginal score per datapoint
# where p_ij is pairwise_pi_wb(pi, deme_config, ndemes, L) -> pi_wb[4]
# and score_der -> pairwise_score(theta, M, ndemes, piw_num, piw_den,...)

def Var_mscore(score_der,p_ij):
    var_mscore = ((score_der^2 * p_ij) - (E_mscore(score_der,p_ij))^2).sum()
    return var_mscore

In [12]:
# Covariance in marginal scores for datapoints at the same locus in different genomes
# (For datapoints at different loci, the covariance is zero)
# where p_ij is pairwise_pi_wb(pi, deme_config, ndemes, L) -> pi_wb[4]
# score_der -> pairwise_score(theta, M, ndemes, piw_num, piw_den,...)
# score_der_b -> pairwise_score(theta, M, ndemes, 0, 0, pib_num, pib_den)
# score_der_w -> pairwise_score(theta, M, ndemes, piw_num, piw_den, 0 ,0)

def cov_mscore(score_der_b, score_der_w, score_der, p_ij):
    cov_mscore = ((score_der_b * score_der_w * p_ij)-(E_mscore(score_der,p_ij))^2).sum()
    return cov_mscore

In [13]:
# Expected composite score
# where p_ij is pairwise_pi_wb(pi, deme_config, ndemes, L) -> pi_wb[4]
# score_der -> pairwise_score(theta, M, ndemes, piw_num, piw_den,...)

def e_composite(score_der,p_ij):
    e_u_1 = L*E_mscore(score_der,p_ij)
    return e_u_1

In [14]:
# Variance in composite score
# where p_ij is pairwise_pi_wb(pi, deme_config, ndemes, L) -> pi_wb[4]
# score_der -> pairwise_score(theta, M, ndemes, piw_num, piw_den,...)
# score_der_b -> pairwise_score(theta, M, ndemes, 0, 0, pib_num, pib_den)
# score_der_w -> pairwise_score(theta, M, ndemes, piw_num, piw_den, 0 ,0)

def var_composite(score_der,score_der_b, score_der_w, theta, mig, p_ij):
    var_u_1 = L*Var_mscore(score_der,p_ij)+L*cov_mscore(score_der_b, score_der_w, p_ij)
    return var_u_1

In [15]:
### Calculate composite likelihood and score from pairwise distance matrix 
### at given parameter values

def calc_cl(pi, deme_config, ndemes, L, theta, M):
    cl = np.zeros(7)
    n = pi.shape[0]
    
    # Compute pairwise distances
    piw_num = 0.0
    piw_den = 0.0
    pib_num = 0.0
    pib_den = 0.0
   
    for i in range(n):
        for j in range(i):
            if(deme_config[i] == deme_config[j]):
                piw_num += pi[i,j]
                piw_den += L
            else:
                pib_num += pi[i,j]
                pib_den += L


    cl[0] = calc_cl(piw_num, piw_den, pib_num, pib_den, theta, M, ndemes)

    # Compute score wrt log of regular parameterization
    eps = 1e-6
    cl[1] = (calc_cl(piw_num, piw_den, pib_num, pib_den, theta*exp(eps/2), M, ndemes)-calc_cl(piw_num, piw_den, pib_num, pib_den, theta*exp(-eps/2), M, ndemes))/eps
    cl[2] = (calc_cl(piw_num, piw_den, pib_num, pib_den, theta, M*exp(eps/2), ndemes)-calc_cl(piw_num, piw_den, pib_num, pib_den, theta, M*exp(-eps/2), ndemes))/eps
    
    # Separately for within demes
    cl[3] = (calc_cl(piw_num, piw_den, 0, 0, theta*exp(eps/2), M, ndemes)-calc_cl(piw_num, piw_den, 0, 0, theta*exp(-eps/2), M, ndemes))/eps
    cl[4] = (calc_cl(piw_num, piw_den, 0, 0, theta, M*exp(eps/2), ndemes)-calc_cl(piw_num, piw_den, 0, 0, theta, M*exp(-eps/2), ndemes))/eps

    # Separately for between demes
    cl[5] = (calc_cl(0, 0, pib_num, pib_den, theta*exp(eps/2), M, ndemes)-calc_cl(0, 0, pib_num, pib_den, theta*exp(-eps/2), M, ndemes))/eps
    cl[6] = (calc_cl(0, 0, pib_num, pib_den, theta, M*exp(eps/2), ndemes)-calc_cl(0, 0, pib_num, pib_den, theta, M*exp(-eps/2), ndemes))/eps
    return cl

## Calculate composite likelihood from sums of pairwise distances at given parameter values


In [16]:
## The super composite log-likelihood for an alignment of n sequences and L sites is then

def calc_cle(pi_wb):
    """
    Calculate composite likelihood estimates from pairwise distance matrix
    
    Parameters
    ----------
    pi : Matrix. The list of sequences in the sample.
    deme_configs : integer
        Number of sequences in the population: 2 * N in a diploid population of N individuals, or N in a
        haploid population of N individuals.
    L : integer
        The length of the sequences
    ndemes : Number of populations
        
    Returns
    -------
    composite log-likelihood for an aligment of n sequences and L length.
    """
    cle = np.zeros(2)
    
    ##piw =  piw_num/piw_den
    piw = pi_wb[0]/pi_wb[1]
    ##pib = pib_num/pib_den
    pib = pi_wb[2]/pi_wb[3]

    
    ## CLEs
    ## Theta
    cle[0] = piw/ndemes
    ## M
    cle[1] = (cle[0]*(ndemes-1))/(pib-piw)

    return cle

In [17]:
#### Parameter estimation

def Etw(M, D):
    """
    Coalescent expectation time for a pair of sequences sampled within
    the same sub-population 
    """
    return D

def Etb(M, D):
    """
    Coalescent expectation time for a pair of sequences sampled between different 
    demes
    """
    return D + (D-1)/M

def Vtw(M, D):
    """
    Coalescent variance time for a pair of sequences sampled within
    the same sub-population 
    """
    return 1/(M+1)+(D-1)/M+(M+1)*kw(M,D)+M*(D-1)*kb(M,D)

def Vtb(M, D):
    """
    Coalescent variance time for a pair of sequences sampled between different 
    demes
    """
    return (D-1)*(M+1)/M/M + (cc(M,D)+1)/(M+1)

def kw(M, D):
    return M*pow(Etb(M,D)/(M+1),2)

def kb(M, D):
    return (D-2)*pow((Etw(M,D)-Etb(M,D))/(D-1),2)

def cc(M, D):
    return (M+1)*(M+1)*(kw(M,D)+(D-1)*kb(M,D))



In [18]:
## Full likelihood for composite 

# def calc_cl_B(pi, deme_config, demes, L, theta, M, n):
def f(x, theta, M):
    #here is the problem
#     print('x', x)
    B = np.exp(x)
#     B = x
#     print(B)
    if(B<1).any():
        B = 1
#     print(theta)
#     print(M)
#     print('B', B)
    ## Define the pairwise likelihoods within demes
    Et = Etw(M, ndemes)
    Vt = Vtw(M, ndemes)
    w00 = np.log(1 - 2*theta*Et + theta*theta/B * (Vt + B*Et*Et))
    w01 = np.log((theta*Et - theta*theta/B) * (Vt + B*Et*Et))
    w11 = np.log(theta*theta/B * (Vt + B*Et*Et))
    ## Between demes
    Et = Etb(M, ndemes)
    Vt = Vtb(M, ndemes)
    b00 = np.log(1 - 2*theta*Et + theta*theta/B * (Vt + B*Et*Et))
    b01 = np.log((theta*Et - theta*theta/B) * (Vt + B*Et*Et))
    b11 = np.log(theta*theta/B * (Vt + B*Et*Et))
    ## Compute the composite log-likelihood for B
    cl = 0.0
    for i in range(n): 
        for j in range(i): 
            X00 = (L-pi[i,j])*(L-pi[i,j]-1)/2
            X01 = (L-pi[i,j])*pi[i,j]
            X11 = pi[i,j]*(pi[i,j]-1)/2
            if(deme_config[i]==deme_config[j]): 
                ## Within demes
                cl += X00*w00 + X01*w01 + X11*w11
            else:
                ## Between demes
                cl += X00*b00 + X01*b01 + X11*b11

    denftr = (L-1)/2*n*(n-1)/2
    super_com = -(cl - denftr)
    super_com.astype(int)
    
    return super_com

In [19]:
## Expected composite log-likelihood -- Driving value

def expcl_deriv(x, theta, M):
    B = np.exp(x)
    if(B<1).any():
        B = 1
    ## Within demes
    Et_w = Etw(M, ndemes)
    Vt_w = Vtw(M, ndemes)
    ## Between demes
    Et_b = Etb(M, ndemes)
    Vt_b = Vtb(M, ndemes)
    ## Compute the expected composite log-likelihood for B (driving value)
    cl_driving = 0.0
    for i in range(n): 
        for j in range(i): 
            if(deme_config[i]==deme_config[j]): 
                ## Within demes
                cl_driving+=(-(pow(theta,2)*((pi[i,j]-L)*(-L+pi[i,j]+1))*Vt_w/2*B*(B*pow((Et_w*theta-1),2)+pow(theta,2)*Vt_w)) +
                            ((pi[i,j]*theta*(pi[i,j]-L)*Vt_w)/B*(B*Et_w*(Et_w*theta+1)+theta*Vt_w)) -
                            (((pi[i,j]-1)*pi[i,j]*Vt_w)/2*B*(B*pow(Et_w,2)+Vt_w)));
            else:
                ## Between demes
                cl_driving+=(-(pow(theta,2)*((pi[i,j]-L)*(-L+pi[i,j]+1))*Vt_b/2*B*(B*pow((Et_b*theta-1),2)+pow(theta,2)*Vt_b)) +
                            ((pi[i,j]*theta*(pi[i,j]-L)*Vt_b)/B*(B*Et_b*(Et_b*theta+1)+theta*Vt_b)) -
                            (((pi[i,j]-1)*pi[i,j]*Vt_b)/2*B*(B*pow(Et_b,2)+Vt_b)));
    
    super_com = -(cl_driving)
    super_com.astype(int)
    ## missing to sum up all the element in the codomain 
    return super_com

In [21]:
## Grid Search and calculation of composite likelihood estimators wrt B (who is this guy)
def calc_cle_B(pi, deme_config, ndemes,L, theta, M):

    n = pi.shape[0]
    mean_pi = np.mean(pi)
    
    gridlen = 20
    logBgrid = np.zeros(gridlen)

    ### Here is the definition of the Grid
    for i in range(gridlen):
        logBgrid[i] = np.log(mean_pi)*i/(gridlen-1)

    init = np.mean(logBgrid)

    raw_funs, polished_x, polished_funs = [], [], []
        
    res = minimize(varcl_deriv, init, method='Nelder-Mead', args = (theta, M), tol = 1.0e-6)
    #res = minimize(f, pt, method='BFGS', args = (theta, M),  tol = 1.0e-8)
    polished_x.append(res.x)
    polished_funs.append(res.fun)

    # evaluation of the solution 
    solution = res['x']
    Bhat = np.exp(solution)
    evaluation = f(solution, theta, M)
    
#     for grid_val in logBgrid:
#         fun_eval = f(grid_val, theta, M)
#         raw_funs.append(fun_eval)
    
    print('Solution: f(%s) = %.5f' % (Bhat, evaluation))
#     print('raw : %s' % raw_funs)
    
    return Bhat

In [22]:
def calc_Epi(deme_config, ndemes, L, theta, M):
    """
    Common population genetics models based on composite likelihood, with well-calibrated
    quantification of uncertainty.
    
    Parameters
    ----------
    pi : Matrix. The list of sequences in the sample.
    ndemes : integer
        Number of sequences in the population: 2 * N in a diploid population of N individuals, or N in a
        haploid population of N individuals.
    L : integer
        The length of the sequences
    ndemes : Number of populations
        
    Returns
    -------
    composite log-likelihood for an aligment of n sequences and L length.
    """

    n = deme_config.shape[0] 
    Epi = np.zeros((samp_size,samp_size)) 
    Epiw = L*theta*ndemes
    Epib = L*theta*(ndemes + (ndemes-1)/M)
    for i in range(n):
        for j in range(i):
            if(deme_config[i]==deme_config[j]):
                Epi[i,j] = Epiw
            else:
                Epi[j,i] = Epib
                            
    return Epi

In [23]:
def calc_error(pi, Epi, deme_config):

    err = np.zeros((2,3)) 
    denom = np.zeros(3)


    for i in range(samp_size):
        for j in range(i):
            
            resid = pi[i,j] - Epi[i,j]
            #print(resid)
            resid2 = (resid)**2
            
            err[0,0] = np.sum(resid)
            err[1,0] = np.sum(resid2)
            
            denom[0] += 1
            
            if(deme_config[i]==deme_config[j]):
                err[0,1] = np.sum(resid)
                err[1,1] = np.sum(resid2)
                denom[1] += 1
            else:
                err[0,2] = np.sum(resid)
                err[1,2] = np.sum(resid2)
                denom[2] += 1

    for k in range(3):
        err[0,k]/=denom[k]
        err[1,k]/=denom[k]

    return err

In [27]:
#### Simulate a dataset

### Set the true parameter values

###Step 1. Global parameters/arguments

ndemes = 2
samp_size_per_deme = 50
samp_size = samp_size_per_deme * ndemes
samp_size_dip = (samp_size_per_deme*2) // 2
N = 1
L = 1e6
seed = 111
B = 1
B2 = 50
M = 0.1 
theta = 0.01
thetaL = (theta*L)
thetaLB = (theta*L)/B
num_reps = 1
simulations = 3
nodes = (2*samp_size)-1
#np.random.seed([42])

In [28]:

### Setting the variables for storing the results

piblock = np.zeros((simulations, samp_size,samp_size))
pibarblock = np.zeros(simulations)
coal_time = np.zeros((samp_size,samp_size))
Et_sim = np.zeros((samp_size,samp_size))
Et2_sim = np.zeros((samp_size,samp_size))

pibar = np.zeros(simulations)
pibar_w = np.zeros(simulations)
pibar_b = np.zeros(simulations)
pi = np.zeros((samp_size,samp_size))

Epi = np.zeros((simulations, samp_size,samp_size)) 
cle_sim = np.zeros((simulations,2))
epi_sim = np.zeros((samp_size,samp_size)) 
cl_B_sim = np.zeros(simulations)
cle_B_sim = np.zeros(simulations)
cle_B_easy_sim = np.zeros(simulations)
cle_sim_mut = np.zeros(simulations)
cle_sim_mig = np.zeros(simulations)

err_sim = np.zeros((simulations,2,3))
data_cle_sim = []
data_cle_B_sim = []
data_cle_B_easy_sim = []
data2_cle_sim = []
pi_b = np.zeros((samp_size,samp_size))

In [29]:
##### TESTING

In [30]:
#### Testing

### testing 
### Simulate a dataset

pi  = np.zeros((simulations, samp_size, samp_size))
clebar_mut = np.zeros(simulations)
clebar_mig = np.zeros(simulations)
p_d_wb = np.zeros((simulations, 5))
test = np.zeros(simulations)
data_cle_sim = []
data_load_msprime = []

for sim in range(simulations):
    
    pi  = np.zeros((simulations, samp_size, samp_size))
    pi_cle = np.zeros((simulations, samp_size, samp_size))
    sims_reps = simulate_tree_MM(samp_size_per_deme, L)
    branch_1 = calc_branches(sims_reps, thetaL)
    branch_load = branch_1[0]
    branch_length = branch_1[1]
    branch_load_msprime = branch_1[2]
    data_load_msprime.append(branch_load_msprime)
    
#     branch_load = np.array(branches['branch_load'])
#     branch_load = np.pad(branch_load, (0, 1), 'constant')
#     print(branch_load)
 
#     display(SVG(sims_reps.draw_svg(size=(700, 500))))
    
#     deme_config = np.zeros(samp_size)
#     for i in range(samp_size):
#         deme_config[i] = i % ndemes
    
    deme_config = []
    for u in sims_reps.nodes():
        deme = u.population
        deme_config.append(deme)

    deme_config = np.array(deme_config[0:samp_size])

## Record per-block pairwise distance matrix and mean pairwise distance
    piblock[sim] = calc_pi(sims_reps, branch_load)
    
    
#     print('piblock: %s' % (np.mean(piblock[sim])))
    pibarblock[sim] = 0.0
    
    for i in range(samp_size):
        for j in range(i):
            pibarblock[sim] += piblock[sim][i,j]*2/samp_size/(samp_size-1)
            
    print(pibarblock)
    #Moments (block-wise to reduce computation)
    coal_time = calc_pi(sims_reps, branch_length)

    for i in range(samp_size):
        for j in range(i):
            Et_sim[i,j] += coal_time[i,j]/2
            Et_sim[j,i] += coal_time[i,j]/2
            Et2_sim[i,j] += coal_time[i,j]*coal_time[i,j]/4
            Et2_sim[j,i] += coal_time[i,j]*coal_time[i,j]/4
    
    print("Second simulation number: {:d}".format(sim))
    
    pi_b = np.zeros((samp_size,samp_size))
    pibar[sim] = 0.0
    pibar_w[sim] = 0.0
    pibar_b[sim] = 0.0
    pibar_w_den = 0.0
    pibar_b_den = 0.0
    
    # Independence between pairs
    for b in range(B2):
        bsim = np.random.randint(int(simulations))
#     b_sim = random.sample(range(samp_size), samp_size)
#     for bsim in random.sample(range(simulations),simulations):
        for i in range(samp_size): 
            for j in range(i):
                pi_b[i,j] += piblock[bsim][i,j]
                pibar[sim] += (piblock[bsim][i,j])*2/samp_size/(samp_size-1)
                if(deme_config[i]==deme_config[j]):
                    pibar_w[sim] += piblock[bsim][i,j] #
                    pibar_w_den += 1
                else:
                    pibar_b[sim] += piblock[bsim][i,j] #
                    pibar_b_den += 1
    
    pibar_w[sim] *= B/pibar_w_den
    pibar_b[sim] *= B/pibar_b_den



    # Symmetric matrix
    for i in range(samp_size):
        for j in range(i):
            pi_b[j,i] = pi_b[i,j]
             
    
    pi = pi_b
#     print(pi)
    pi_cle = calc_pi(sims_reps, branch_load)
    n = pi.shape[0]
    
    p_d_wb = pairwise_pi_wb(piblock[sim],deme_config,ndemes,L)
    p_ij = p_d_wb[4]

    cle_sim = calc_cle(p_d_wb)
    data_cle_sim.append(cle_sim)
    print(cle_sim)
    
    cle_sim_mut = cle_sim[0]
    cle_sim_mig = cle_sim[1]
#     print(cle_sim_mut)
    
    test = full_loglikelihood(p_ij)
    
    
    cle_B_sim = calc_cle_B(pi,deme_config,ndemes,L,cle_sim_mut,cle_sim_mig)
#     data_b = pd.DataFrame(data_cle_B_sim.append(cle_B_sim))
    data_cle_B_sim.append(cle_B_sim)
    
#     cle_B_easy_sim = calc_cle_B(pi,deme_config,ndemes,L,0.01,0.1)
# #     data_b_easy = pd.DataFrame(data_cle_B_easy_sim.append(cle_B_easy_sim))
#     data_cle_B_easy_sim.append(cle_B_easy_sim)

data_cleb = pd.DataFrame(data_cle_sim)
data_b = pd.DataFrame(data_cle_B_sim)
data_b_easy = pd.DataFrame(data_cle_B_easy_sim)

# data_cleB_sim = pd.concat([data_cleb, data_b, data_b_easy], axis=1)
# writer = pd.ExcelWriter('cleB_050822.xlsx')
# data_cleB_sim.to_excel(writer,'Sheet1')
# writer.save()
    

print(data_cle_sim)

print('pibars')
print(pibar_w, pibar_b, pibar)


# data_cle_sim = pd.DataFrame(data_cle_sim)
# data_load_msprime = pd.DataFrame(data_load_msprime)

# data_cle_sim = pd.concat([data_cle_sim, data_load_msprime], axis=1)
# writer = pd.ExcelWriter('cle.xlsx')
# data_cle_sim.to_excel(writer,'Sheet1')
# writer.save()

# plt.hist(data_cle_sim[0])
Epi = calc_Epi(deme_config, ndemes, L, theta, M)
calc_error(pi, Epi, deme_config)

[58796.05656566     0.             0.        ]
Second simulation number: 0
[0.02159549 0.69892672]


/var/folders/x5/vb2nw9156tq5vyxycxy87m31mqdc_q/T/ipykernel_73894/3713639727.py:19: RuntimeWarning: overflow encountered in multiply
  (((pi[i,j]-1)*pi[i,j]*Vt_w*(2*B*pow(Et_w,2)+Vt_w))/2*pow(B,2)*pow((B*pow(Et_w,2)+Vt_w),2)));
/var/folders/x5/vb2nw9156tq5vyxycxy87m31mqdc_q/T/ipykernel_73894/3713639727.py:17: RuntimeWarning: overflow encountered in multiply
  var_cl+=((pow(theta,2)*(L-pi[i,j]-1)*(L-pi[i,j])*Vt_w*(2*B*pow(Et_w*theta-1,2)+pow(theta,2)*Vt_w)/2*pow(B,2)*pow((B*pow((Et_w*theta-1),2)+pow(theta,2)*Vt_w),2)) +
/var/folders/x5/vb2nw9156tq5vyxycxy87m31mqdc_q/T/ipykernel_73894/3713639727.py:24: RuntimeWarning: overflow encountered in multiply
  (((pi[i,j]-1)*pi[i,j]*Vt_b*(2*B*pow(Et_b,2)+Vt_b))/2*pow(B,2)*pow((B*pow(Et_b,2)+Vt_b),2)));
/var/folders/x5/vb2nw9156tq5vyxycxy87m31mqdc_q/T/ipykernel_73894/3713639727.py:22: RuntimeWarning: overflow encountered in multiply
  var_cl+=((pow(theta,2)*(L-pi[i,j]-1)*(L-pi[i,j])*Vt_b*(2*B*pow(Et_b*theta-1,2)+pow(theta,2)*Vt_b)/2*pow(B,2)*pow((B

Solution: f([7.18335163e+59]) = 375422031743957440.00000
[58796.05656566 99494.9389899      0.        ]
Second simulation number: 1
[0.0036565  0.02003339]
Solution: f([1.67489828e+65]) = 8694750752210656256.00000
[58796.05656566 99494.9389899  21149.04222222]
Second simulation number: 2
[0.00500625 0.22703689]
Solution: f([1.55575108e+65]) = 7932627053256153088.00000
[array([0.02159549, 0.69892672]), array([0.0036565 , 0.02003339]), array([0.00500625, 0.22703689])]
pibars
[12957.29142857 19191.50883265 22474.03636735] [ 22226.712  100290.4664  96204.446 ] [ 881940.84848485 3007528.91555554 2985580.84949492]


array([[5.38666667e+02, 1.08832653e+03, 1.66479800e+03],
       [1.43630080e+09, 2.90191386e+09, 6.92888095e+09]])

In [31]:
print(cle_sim)
print(p_d_wb)
print(test)
print(data_b)
# print(data_b_easy)

[0.00500625 0.22703689]
[2.45306090e+07 2.45000000e+09 8.01571500e+07 2.50000000e+09
 4.20753535e-02]
3.168293138004423
              0
0  7.183352e+59
1  1.674898e+65
2  1.555751e+65


In [32]:
calc_Epi(deme_config, ndemes, L, theta, M)

array([[     0.,      0.,      0., ..., 120000., 120000., 120000.],
       [ 20000.,      0.,      0., ..., 120000., 120000., 120000.],
       [ 20000.,  20000.,      0., ..., 120000., 120000., 120000.],
       ...,
       [     0.,      0.,      0., ...,      0.,      0.,      0.],
       [     0.,      0.,      0., ...,  20000.,      0.,      0.],
       [     0.,      0.,      0., ...,  20000.,  20000.,      0.]])